In [16]:
import random
import time
import pynq
from pynq import Overlay
import numpy as np
from pynq import Clocks
from pynq import allocate
import sys
import struct

def LDL_dsolve(n, X, D, Xout_sw):
    for i in range(n):
        Xout_sw[i] = X[i] / D[i]

def generate_random_array(n, min_value, max_value):
    return [random.uniform(min_value, max_value) for _ in range(n)]

def round_array(arr, decimals):
    return [round(num, decimals) for num in arr]

In [ ]:
n = 10000

X_sw = generate_random_array(n, 0.5, 500.0)

X_sw = round_array(X_sw, 7)


D_sw = generate_random_array(n, 0.5, 500.0)

D_sw = round_array(D_sw, 7)

Xout_sw = [0.0] * n

392.2728861


In [19]:
start_time_sw = time.time()
LDL_dsolve(n, X_sw, D_sw, Xout_sw)
end_time_sw = time.time()
software_time = end_time_sw - start_time_sw
print(software_time) #software time measured

0.9289624691009521


In [22]:
alveare = Overlay("doublerate.bit")
# alveare?
ape = alveare.ldl_dsolve_0

padding = False

if(n % 2 == 1) #if the number of elements is odd, add one more element to make it even and set it to 1.0 later
    n++
    padding = True


X_hw = pynq.allocate(n, np.float64)
D_hw = pynq.allocate(n, np.float64)
Xout_hw = pynq.allocate(n, np.uint64)

X_hw[:] = X_sw
D_hw[:] = D_sw

X_hw[-1] = 1.0
D_hw[-1] = 1.0

#communicate the buffer address
ape.register_map

ape.write(0x18, X_hw.physical_address)
ape.write(0x24, D_hw.physical_address)
ape.write(0x30, Xout_hw.physical_address)
ape.write(0x10, n)

X_hw.physical_address

# ape.read(0x10)
# ape.read(0x18)
# ape.read(0x24)
# ape.read(0x30)

OSError: Bitstream file doublerate.bit does not exist.

In [21]:
start_time_hw = time.time()
#start
ape.write(0x00, 0x01)
#polling
while ape.read(0x00) & 0x4 != 4:
    pass
#read back
Xout_hw.sync_from_device()
end_time_hw = time.time()
hardware_time = end_time_hw - start_time_hw
print(hardware_time) #hardware time measured

# bin(struct.unpack('!i',struct.pack('!f',Xout_hw[1]))[0])

0.009105682373046875


In [ ]:
if(hardware_time != 0) {
    speedup = software_time/hardware_time
}

print(speedup)

In [14]:
Xfinal = [0.0]
if(padding):
    n--
for i in range(n):
    uint64_bytes = struct.pack('Q', Xout_hw[i])
    # Unpack bytes as double
    double_value = struct.unpack('d', uint64_bytes)[0]
    Xfinal.append(double_value)
Xfinal

[0.0, 2.208122721500743]

In [15]:
Xout_sw[0]

2.208122721500743